In [ ]:
import hjson
import folium

In [ ]:
#Calculate distance from latlon
# https://qiita.com/s-wakaba/items/e12f2a575b6885579df7

from math import sin, cos, acos, radians
earth_rad = 6378.137

def latlng_to_xyz(lat, lng):
    rlat, rlng = radians(lat), radians(lng)
    coslat = cos(rlat)
    return coslat*cos(rlng), coslat*sin(rlng), sin(rlat)

def dist_on_sphere(pos0, pos1, radious=earth_rad):
    xyz0, xyz1 = latlng_to_xyz(*pos0), latlng_to_xyz(*pos1)
    return acos(sum(x * y for x, y in zip(xyz0, xyz1)))*radious

Osaka = 34.702113, 135.494807
Tokyo = 35.681541, 139.767103
London = 51.476853, 0.0

print(dist_on_sphere(Osaka, Tokyo)) # 403.63km
print(dist_on_sphere(London, Tokyo)) # 9571.22km

In [ ]:
#init
map = folium.Map(location=[35.170833333333334, 138.18125], zoom_start=4)

#load
with open('dataset.hjson', 'r') as f:
    dataset = hjson.load(f)

#save map
for pref, v in dataset.items():
    lat0 = v['lat0']
    lat1 = v['lat1']
    lon0 = v['lon0']
    lon1 = v['lon1']
    
    if pref in ('akita', 'yamanashi', 'niigata'):
        color = 'red'
    else:
        color = 'black'
        
    #distance
    dist_lat = int(dist_on_sphere([lat0, lon0], [lat1, lon0]))
    dist_lon = int(dist_on_sphere([lat0, lon0], [lat0, lon1]))
    
    #pop-up
    popup = f'{pref} distance_lat={dist_lat} dist_lon={dist_lon}'
    
    #polyline
    folium.PolyLine(
        ([lat0, lon0], [lat0, lon1], [lat1, lon1], [lat1, lon0], [lat0, lon0]),
        popup=popup,
        color=color).add_to(map)
    
    #circle marker
    folium.CircleMarker(
        location=[lat0, lon0],
        radius=1,
        color='#3186cc', fill_color='#3186cc',
    ).add_to(map)

map.save(f'/tmp/map.html')
map